In [ ]:
from pyAudioAnalysis import audioBasicIO as aIO;
from pyAudioAnalysis import audioSegmentation as aS;
from pydub import AudioSegment;
from pydub.silence import split_on_silence;
import os;
import pandas as pd;
import numpy as np;

In [ ]:
# Create new folders for the clips with removed silence
# Folders created within the Data folder

if not os.path.isdir('../Data/Train_no_silence'):
    os.mkdir('../Data/Train_NoSilence')
    
if not os.path.isdir('../Data/Test_no_silence'):
    os.mkdir('../Data/Test_NoSilence')

In [ ]:
# Getting all the files from the training set
files_train = os.listdir('Train')
files_test = os.listdir('Test')

# sorting the files by name
files_train.sort()
files_test.sort()


# I'm using MacOS, and it by default has this .DS_store file, which I can't get rid of. 
# It's the first file in the folder.
# That's why I'm removing it. If you are using this code in your own PC, this might not apply to you.
# in that case, comment out the del function line.
del files_train[0] 

In [ ]:
# add path name to the file name
fnames_train = []
fnames_test  = []
for i in range(len(files_train)):
    fname = "Train/%s" % files_train[i]
    fnames_train.append(fname)
    
for i in range(len(files_test)):
    fname = "Test/%s" % files_test[i]
    fnames_test.append(fname)

# sort the list of file names with path     
fnames_train.sort()
fnames_test.sort()

In [ ]:
df_test = pd.read_csv('Categories_test.csv')
df_train = pd.read_csv('Categories_train.csv')

fnames_test_df = pd.DataFrame(files_test, columns=['FileID'])
fnames_train_df = pd.DataFrame(files_train, columns=['FileID'])


only_fileID = pd.DataFrame(df_train['FileID'])

In [ ]:
#defining a function that turns the frame segment from seconds to miliseconds, because pydub operates on milisecs.
# It also depends on how many slices are returned. 

def turn_mSec(segments):
    
    if len(segments)==1:
        segments[0][0]=segments[0][0]*1000
        segments[0][1]=segments[0][1]*1000
    elif len(segments)==2:
        segments[0][0]=segments[0][0]*1000
        segments[0][1]=segments[0][1]*1000
        segments[1][0]=segments[1][0]*1000
        segments[1][1]=segments[1][1]*1000
    elif len(segments)==3:
        segments[0][0]=segments[0][0]*1000
        segments[0][1]=segments[0][1]*1000
        segments[1][0]=segments[1][0]*1000
        segments[1][1]=segments[1][1]*1000
        segments[2][0]=segments[2][0]*1000
        segments[2][1]=segments[2][1]*1000
    elif len(segments)==4:
        segments[0][0]=segments[0][0]*1000
        segments[0][1]=segments[0][1]*1000
        segments[1][0]=segments[1][0]*1000
        segments[1][1]=segments[1][1]*1000
        segments[2][0]=segments[2][0]*1000
        segments[2][1]=segments[2][1]*1000 
        segments[3][0]=segments[3][0]*1000
        segments[3][1]=segments[3][1]*1000    
    elif len(segments)==5:
        segments[0][0]=segments[0][0]*1000
        segments[0][1]=segments[0][1]*1000
        segments[1][0]=segments[1][0]*1000
        segments[1][1]=segments[1][1]*1000
        segments[2][0]=segments[2][0]*1000
        segments[2][1]=segments[2][1]*1000 
        segments[3][0]=segments[3][0]*1000
        segments[3][1]=segments[3][1]*1000
        segments[4][0]=segments[4][0]*1000
        segments[4][1]=segments[4][1]*1000
    
    return segments;

In [ ]:
#defining a function to get the indicated segments, slice the sound, and then combine those slices.

def get_combine_slices(segments, sound):

    if len(segments)==1:
        slice1 = sound[segments[0][0]:segments[0][1]]
        final_sound = slice1
    elif len(segments)==2:
        slice1 = sound[segments[0][0]:segments[0][1]]
        slice2 = sound[segments[1][0]:segments[1][1]]
        final_sound = slice1+slice2
    elif len(segments)==3:
        slice1 = sound[segments[0][0]:segments[0][1]]
        slice2 = sound[segments[1][0]:segments[1][1]]
        slice3 = sound[segments[2][0]:segments[2][1]]
        final_sound = slice1+slice2+slice3
    elif len(segments)==4:
        slice1 = sound[segments[0][0]:segments[0][1]]
        slice2 = sound[segments[1][0]:segments[1][1]]
        slice3 = sound[segments[2][0]:segments[2][1]]
        slice4 = sound[segments[3][0]:segments[3][1]]
        final_sound = slice1+slice2+slice3+slice4
    elif len(segments)==4:
        slice1 = sound[segments[0][0]:segments[0][1]]
        slice2 = sound[segments[1][0]:segments[1][1]]
        slice3 = sound[segments[2][0]:segments[2][1]]
        slice4 = sound[segments[3][0]:segments[3][1]]
        slice5 = sound[segments[4][0]:segments[4][1]]
        final_sound = slice1+slice2+slice3+slice4+slice5
    elif len(segments)==4:
        slice1 = sound[segments[0][0]:segments[0][1]]
        slice2 = sound[segments[1][0]:segments[1][1]]
        slice3 = sound[segments[2][0]:segments[2][1]]
        slice4 = sound[segments[3][0]:segments[3][1]]
        slice5 = sound[segments[4][0]:segments[4][1]]
        slice6 = sound[segments[4][0]:segments[4][1]]
        final_sound = slice1+slice2+slice3+slice4+slice5+slice6
        

    return final_sound



In [ ]:
##Try out

Fs, signal = aIO.read_audio_file(fnames_train[383]);
segments = aS.silence_removal(signal, Fs, 0.020, 0.020, smooth_window = 0.4, weight = 0.4, plot = False);
segments = turn_mSec(segments);  
sound = AudioSegment.from_file(fnames_train[i]);

print(segments)
print(sound)

In [ ]:
# Getting new short clips and saving them on file. This is for the train set. 
    
pathname = "Train_NoSilence/";
oldDur = [];
newDur = [];


for i in np.arange(0,len(fnames_train),1):    
    print(i)
    Fs, signal = aIO.read_audio_file(fnames_train[i]);
    segments = aS.silence_removal(signal, Fs, 0.020, 0.020, smooth_window = 0.4, weight = 0.4, plot = False);
    segments = turn_mSec(segments);
    
    sound = AudioSegment.from_file(fnames_train[i]);
    
    final_sound = get_combine_slices(segments, sound);
    
    new_fname_path = pathname + files_train[i]
    output_file = "{}.wav".format(new_fname_path)
    print("Exporting file", output_file)
    final_sound.export(output_file, format="wav")
    
    old_dur = sound.duration_seconds
    new_dur = final_sound.duration_seconds
    
    oldDur.append(old_dur);
    newDur.append(new_dur);

In [ ]:
# Getting new short clips and saving them on file. This is for the test set. 
    
pathname_t = "Test_NoSilence/";
oldDur_Test = [];
newDur_Test = [];


for i in np.arange(0,len(fnames_test,1):    
    Fs, signal = aIO.read_audio_file(fnames_test[i]);
    segments = aS.silence_removal(signal, Fs, 0.020, 0.020, smooth_window = 0.4, weight = 0.4, plot = False);
    
    segments = turn_mSec(segments);
    sound = AudioSegment.from_file(fnames_test[i]);
    final_sound = get_combine_slices(segments, sound);
    
    new_fname_path = pathname + files_test[i]
    output_file = "{}.wav".format(new_fname_path)
    print("Exporting file", output_file)
    final_sound.export(output_file, format="wav")
    
    old_dur = sound.duration_seconds
    new_dur = final_sound.duration_seconds
    
    oldDur_Test.append(old_dur);
    newDur_Test.append(new_dur);

In [ ]:
df_train = pd.DataFrame(columns = ['FileID', 'ActorID', 'SentenceID', 'Emotion', 'Intensity'])

for file in files_train:
    fileID = file.split('.')[0]
    cats = fileID.split('_')
    
    new_row = pd.DataFrame([[fileID] + cats],
                           columns = ['FileID', 'ActorID', 'SentenceID', 'Emotion', 'Intensity'])
    df_train = pd.concat((df_train,new_row), axis=0, ignore_index=True)

# Turn columns into factors (except for FileID)
for col in df_train.columns:
    if col != 'FileID':
        df_train[col] = (df_train[col]).astype('category')

In [ ]:
df_train['Old_Dur'] = oldDur
df_train['New_Dur'] = newDur
df_train

In [ ]:


df_test = pd.DataFrame(columns = ['FileID', 'ActorID', 'SentenceID', 'Emotion', 'Intensity'])

for file in files_test:
    fileID = file.split('.')[0]
    cats = fileID.split('_')
    
    new_row = pd.DataFrame([[fileID] + cats],
                        columns = ['FileID', 'ActorID', 'SentenceID', 'Emotion', 'Intensity'])
    df_test = pd.concat((df_test,new_row), axis=0, ignore_index=True)

# Turn columns into factors (except for FileID)
for col in df_test.columns:
    if col != 'FileID':
        df_test[col] = (df_test[col]).astype('category')

In [ ]:
df_test['Old_Dur'] = oldDur_Test
df_test['New_Dur'] = newDur_Test
df_test

In [ ]:
# # Removing silence for Test data
# pathname_t = "Test_NoSilence/";
# oldDur_Test = [];
# newDur_Test = [];

# # Note the index starts from 0 in this loop, because there's no .DS_store file in this folder.
# for i in np.arange(0,len(fnames_test),1):
#     print(i)
#     t1, t2 = remove_silence(fnames_test[i], files_test[i], pathname_t);
#     oldDur_Test.append(t1);
#     newDur_Test.append(t2);    